In [25]:
import json

In [26]:
def generate_machine(machine_id, base_power, switching_off_power, switching_on_power, sleeping_power, switching_on_time, switching_off_time):
    """Generate one machine entry with DVFS profiles and states."""

    base_speed = 1.0
    underclock_power = base_power * 0.7
    underclock_speed = base_speed * 0.7
    overclock_power = base_power * 1.3
    overclock_speed = base_speed * 1.3
    
    machine = {
        "id": machine_id,
        "dvfs_profiles": {
            "underclock_1": {"power": underclock_power, "compute_speed": underclock_speed},
            "base": {"power": base_power, "compute_speed": base_speed},
            "overclock_1": {"power": overclock_power, "compute_speed": overclock_speed},
        },
        "dvfs_mode": "base",
        "states": {
            "active": {
                "power": "from_dvfs",
                "compute_speed": "from_dvfs",
                "can_run_jobs": True,
                "transitions": [{"state": "switching_off", "transition_time": 0}],
            },
            "switching_off": {
                "power": switching_off_power,
                "compute_speed": "from_dvfs",
                "can_run_jobs": False,
                "transitions": [{"state": "sleeping", "transition_time": switching_off_time}],
            },
            "switching_on": {
                "power": switching_on_power,
                "compute_speed": "from_dvfs",
                "can_run_jobs": False,
                "transitions": [{"state": "active", "transition_time": switching_on_time}],
            },
            "sleeping": {
                "power": sleeping_power,
                "compute_speed": "from_dvfs",
                "can_run_jobs": False,
                "transitions": [{"state": "switching_on", "transition_time": 0}],
            },
        },
    }

    return machine


def generate_cluster(num_node):
    machines = [
        generate_machine(machine_id, 190, 9, 190, 9, 120, 120) for machine_id in range(num_node)
    ]

    return {"machines": machines}




In [27]:
# Number of nodes to generate
num_node = 32
# Path to output JSON
output_path = "platforms/generated_32.json"


In [28]:

cluster = generate_cluster(num_node)

with open(output_path, "w") as json_file:
    json.dump(cluster, json_file, indent=4)
